In [1]:
pip install tensorflow numpy opencv-python


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionV3


In [3]:
# Load pre-trained I3D model (using InceptionV3 as a substitute for feature extraction)
base_model = InceptionV3(weights='imagenet', include_top=False, pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
def load_and_preprocess_images(folder_path, target_size=(224, 224)):
    images = []
    for img_name in sorted(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale (optical flow)
        if img is None:
            continue
        img = cv2.resize(img, target_size)  # Resize
        img = np.expand_dims(img, axis=-1)  # Add channel dimension
        img = np.repeat(img, 3, axis=-1)  # Convert to 3 channels for I3D
        img = img.astype(np.float32) / 255.0  # Normalize
        images.append(img)
    
    return np.array(images)


In [ ]:
# Path to dataset
dataset_path = "/kaggle/input/shanghaitech-anomaly-detection/dataset/mp"  # Updated path

# Dictionary to store optical flow features
optical_flow_features = {}

for video_folder in sorted(os.listdir(dataset_path)):
    flow_path = os.path.join(dataset_path, video_folder, "optical_flow")  # Optical Flow folder

    if not os.path.exists(flow_path):
        continue

    print(f"Processing Optical Flow for {video_folder}...")

    flow_images = load_and_preprocess_images(flow_path)  # You already defined this elsewhere

    if len(flow_images) == 0:
        print(f"Skipping {video_folder}, no valid optical flow images found.")
        continue

    # Extract features
    features = model.predict(flow_images, batch_size=16)  # model defined in another cell

    # Store features
    optical_flow_features[video_folder] = features

# Save extracted optical flow features
np.save("i3d_optical_flow_features.npy", optical_flow_features)
print("Optical Flow Feature extraction completed! 🚀")


Processing Optical Flow for 01_001...
48/48 ━━━━━━━━━━━━━━━━━━━━ 16s 169ms/step
Processing Optical Flow for 01_002...
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
Processing Optical Flow for 01_003...
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 134ms/step
Processing Optical Flow for 01_004...
 1/48 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step

In [ ]:
# Dictionary to store RGB frame features
rgb_frame_features = {}

for video_folder in sorted(os.listdir(dataset_path)):
    frame_path = os.path.join(dataset_path, video_folder, "frames")  # RGB Frames folder
    
    if not os.path.exists(frame_path):
        continue

    print(f"Processing RGB Frames for {video_folder}...")
    
    frame_images = load_and_preprocess_images(frame_path)

    if len(frame_images) == 0:
        print(f"Skipping {video_folder}, no valid RGB frames found.")
        continue

    # Extract features
    features = model.predict(frame_images, batch_size=16)
    
    # Store features
    rgb_frame_features[video_folder] = features

# Save extracted RGB frame features
np.save("i3d_rgb_frame_features.npy", rgb_frame_features)
print("RGB Frame Feature extraction completed! 🚀")


In [ ]:
# Load extracted features
optical_flow_features = np.load("i3d_optical_flow_features.npy", allow_pickle=True).item()
rgb_frame_features = np.load("i3d_rgb_frame_features.npy", allow_pickle=True).item()

# Function to calculate similarity
def cosine_similarity(feature1, feature2):
    dot_product = np.dot(feature1, feature2.T)
    norm1 = np.linalg.norm(feature1, axis=1, keepdims=True)
    norm2 = np.linalg.norm(feature2, axis=1, keepdims=True)
    similarity = dot_product / (norm1 * norm2.T)
    return np.mean(similarity)

# Compare features for each video
similarity_scores = {}

for video_folder in optical_flow_features.keys():
    if video_folder in rgb_frame_features:
        sim_score = cosine_similarity(optical_flow_features[video_folder], rgb_frame_features[video_folder])
        similarity_scores[video_folder] = sim_score
        print(f"Similarity Score for {video_folder}: {sim_score:.4f}")

# Save similarity scores
np.save("i3d_similarity_scores.npy", similarity_scores)
print("Feature comparison completed! ✅")
